# KDDM Group 17


## 0: Package and data Import 

Importing and load necessary packages

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)

## 1: Data Understanding

Set directory and load data

In [2]:
# Change the current working directory
os.chdir("C:/Users/schra/OneDrive/Dokumente/CSS/2. Semester/KDDM/Group Project, 17")
# Reading the CSV file into a DataFrame
df = pd.read_csv('data_7.csv')

Overview and summary of the data

In [ ]:
df.shape
#print(df.head())
#print(df.info())

# Summary Statistics
print(df.describe())

# Exportiert summary Tabelle
#summary_stats = df.describe()
#summary_stats.to_excel('summary_statistics.xlsx', index=True)


## 2: Data Preparation

Handling Missing data

In [3]:
# Handling Missing Values
#print(df.isna().sum())

#Drop na (HeatingType), removes the 5 missing values across all values
df = df.dropna(how='all').copy()
#print(df.isna().sum())
print(df.head())

#Remove all missing data
# !! Not for Pool
#df = df.dropna().copy()

   Bedrooms  Bathrooms  SquareFootageHouse   Location   Age PoolQuality  \
0       1.0        1.0                46.0  Suburbann  68.0         NaN   
1       1.0        1.0                88.0  Suburbann  20.0         NaN   
2       NaN        1.0               122.0      Urban  40.0         NaN   
3       NaN        2.0               112.0      Rural  69.0         NaN   
4       1.0        NaN                42.0        NaN  16.0   Excellent   

  HasPhotovoltaics  HeatingType HasFiberglass IsFurnished DateSinceForSale  \
0              NaN          Gas         False        True       2024-02-04   
1             True     Electric         False        True       2024-01-31   
2              NaN          Gas         False        True       2023-08-30   
3            False  Electricity          True        True       2023-03-18   
4            False  Oil Heating         False       False       2023-07-26   

  HouseColor PreviousOwnerName HasFireplace KitchensQuality BathroomsQuality  \


In [4]:
df.describe()

,Bedrooms,Bathrooms,SquareFootageHouse,Age,SquareFootageGarden,PreviousOwnerRating,HeatingCosts,Price
count,618.000000,630.000000,995.000000,868.000000,995.000000,995.000000,516.000000,995.000000
mean,1.621359,1.658730,175.727638,43.324885,11.538693,2.576905,125.134496,253.826843
std,0.941909,0.922796,821.254214,18.770587,7.898836,0.894898,507.796163,428.592868
min,1.000000,1.000000,-895.000000,-94.000000,6.000000,1.000000,-443.000000,29.123820
25%,1.000000,1.000000,53.000000,33.000000,6.000000,1.870000,49.000000,128.396785
50%,1.000000,1.000000,86.000000,45.000000,6.000000,2.490000,76.900000,188.658440
75%,2.000000,2.000000,116.000000,57.000000,18.000000,3.165000,127.650000,270.963420
max,5.000000,5.000000,8578.000000,71.000000,30.000000,4.970000,8292.000000,6050.591600


Data type transformation

In [ ]:
# Location
# There are 4 instead of 3 categories:
# ['Rural', 'Suburban', 'Suburbann', 'Urban'] 
# The typo will be corrected and the data summarized
df['Location'].replace('Suburbann', 'Suburban', inplace=True)
pd.Categorical(df['Location'])

# Heating Type
# There are 5 instead of 3 categories:
# ['Electric', 'Electricity', 'Gas', 'Oil', 'Oil Heating'] 
# The data will be summarized
# ['Electric', 'Gas', 'Oil']
df['HeatingType'].replace('Electricity', 'Electric', inplace=True)
df['HeatingType'].replace('Oil Heating', 'Oil', inplace=True)
pd.Categorical(df['HeatingType'])

Test for categories

In [ ]:
# Pool Quality
# 3 categories:
# ['Excellent', 'Good', 'Poor']
pd.Categorical(df['PoolQuality'])

# Has Photovoltaics 
# Bool
# [False, True]
pd.Categorical(df['HasPhotovoltaics'])

# Heating Type 
# 3 categories:
# ['Excellent', 'Good', 'Poor']
pd.Categorical(df['HeatingType'])

# Has Fiberglass
# Bool
# [False, True]
pd.Categorical(df['HasFiberglass'])

# Is Furnished
# Bool
# [False, True]
pd.Categorical(df['IsFurnished'])

# House Color 
# 4 categories:
# ['Gray', 'Green', 'White', 'Yellow']
pd.Categorical(df['HouseColor'])

# Has Fireplace 
# Bool
# [False, True]
pd.Categorical(df['HasFireplace'])

# Kitchens Quality 
# 3 categories:
# ['Excellent', 'Good', 'Poor']
pd.Categorical(df['KitchensQuality'])

# Bathrooms Quality 
# 3 categories:
# ['Excellent', 'Good', 'Poor']
pd.Categorical(df['BathroomsQuality'])

# Bedrooms Quality 
# 3 categories:
# ['Excellent', 'Good', 'Poor']
pd.Categorical(df['BedroomsQuality'])

# Livingrooms Quality 
# 3 categories:
# ['Excellent', 'Good', 'Poor']
pd.Categorical(df['LivingRoomsQuality'])

# Previous Ownder Rating
# 1.00 to 4.97
# [1.00, 1.01, 1.02, 1.03, ..., 4.93, 4.94, 4.96, 4.97]
pd.Categorical(df['PreviousOwnerRating'])

# Window Model Names
# Way too many categories
# 995
# Each house has its own category (No duplicates)
pd.Categorical(df['WindowModelNames'])
print(len(df['WindowModelNames']))
duplicates_mask = df['WindowModelNames'].duplicated()
duplicates_df = df[duplicates_mask]
print("Duplicate rows:")
print(duplicates_df)

Test for duplicates

In [ ]:
df.loc[df.duplicated()]
# No duplicate rows 

Output of all cases with SquareFootageHouse <0

In [ ]:
# Count the cases where 'SquareFootageHouse       ' is smaller than 0
negative_square_footage_house_count = (df['SquareFootageHouse'] < 0).sum()
# Display the count
print("Number of cases where SquareFootageHouse is smaller than 0:", negative_square_footage_house_count)

# Filter the DataFrame to get rows where 'SquareFootageHouse' is smaller than 0
negative_square_footage_house_cases = df[df['SquareFootageHouse'] < 0]
# Print the cases where 'SquareFootageHouse       ' is smaller than 0
print("Cases where SquareFootageHouse  is smaller than 0:")
print(negative_square_footage_house_cases)

# There are 8 cases
# I would suggest to simply make them positive (Could be just a typo)
# Everything else seems okay

Output of all cases with Age <0

In [ ]:
# Count the cases where 'Age' is smaller than 0
negative_age_house_count = (df['Age'] < 0).sum()
# Display the count
print("Number of cases where Age is smaller than 0:", negative_age_house_count)

# Filter the DataFrame to get rows where 'Age' is smaller than 0
negative_age_house_cases = df[df['Age'] < 0]
# Print the cases where 'Age' is smaller than 0
print("Cases where Age  is smaller than 0:")
print(negative_age_house_cases)

# There are 12 cases
# I would suggest to simply make them positive (Could be just a typo)
# Everything else seems okay

Output of all cases with HeatingCost <0

In [ ]:
# Count the cases where 'HeatingCost' is smaller than 0
negative_heating_cost_count = (df['HeatingCosts'] < 0).sum()
# Display the count
print("Number of cases where Age is smaller than 0:", negative_heating_cost_count)

# Filter the DataFrame to get rows where 'HeatingCost' is smaller than 0
negative_heating_cost_cases = df[df['HeatingCosts'] < 0]
# Print the cases where 'HeatingCost' is smaller than 0
print("Cases where HeatingCost  is smaller than 0:")
print(negative_heating_cost_cases)

# There are 5 cases 
# I would suggest to simply make them positive (Could be just a typo)
# Everything else seems okay

Comparison of Age and DateSinceForSale

In [ ]:
# Extracts the Year since the house is for sale
df['Year'] = df['DateSinceForSale'].str.extract(r'(\d{4})')
# Calculates the Year, the house was build in
df['NewAge'] =  2024 - df['Age'] 

# Subtracts Age the house was build in from the Year since sale, to look for anomalies
result = df[df['Year'] < df['NewAge']]
print(result)

# Shows no errors/anomalies in the code, expect the cases with negative Age 
# and houses that were build this year and were put up for sale

Calculating General Square Footage

In [ ]:
#Overall Square Footage
# Negative Outliners need to be corrected first!
df['seneral_square_footage'] = df['SquareFootageHouse'] + df['SquareFootageGarden']
#print(df['seneral_square_footage'].describe())


Calculating Room Count

In [ ]:
# Overall Room Count 
# Negative Outliners need to be corrected first! 
df['Rooms'] = df['Bedrooms'] + df['Bathrooms'] 
#print(df['Rooms'].describe()) 
#df['Rooms'] 

Calculating Overall House Quality

In [8]:
# Overall House Quality
# Quality Categories need to be converted to numeric values
category_map = {'Excellent': 3, 'Good': 2, 'Poor': 1}

# Convert categorical values to numerical values
df["PoolQuality"] = df["PoolQuality"].replace(category_map)
df["KitchensQuality"] = df["KitchensQuality"].replace(category_map)
df["BathroomsQuality"] = df["BathroomsQuality"].replace(category_map)
df["BedroomsQuality"] = df["BedroomsQuality"].replace(category_map)
df["LivingRoomsQuality"] = df["LivingRoomsQuality"].replace(category_map)

# Calculate average score across columns
df["overall_quality"] = df["PoolQuality"] + df["KitchensQuality"] + df["BathroomsQuality"] + df["BedroomsQuality"] + df["LivingRoomsQuality"] 
#print(df["overall_quality"].describe())

# Count the number of non-missing values for each row
num_non_missing_values = df[["PoolQuality", "KitchensQuality", "BathroomsQuality", "BedroomsQuality", "LivingRoomsQuality"]].notnull().sum(axis=1)
#num_non_missing_values.describe()

# Calculate the average overall quality score
df["average_overall_quality"] = df["overall_quality"] / num_non_missing_values.replace(0, np.nan)  # Replace 0 with NaN to handle division by zero

# Print summary statistics of the average overall quality score
#print(df["average_overall_quality"].describe())

count    995.000000
mean       4.353769
std        0.478379
min        4.000000
25%        4.000000
50%        4.000000
75%        5.000000
max        5.000000
dtype: float64

## 3: